In [1]:
import pandas as pd
import numpy as np
import torchtext 
import torch
from sklearn.model_selection import train_test_split

In [2]:
data_train = pd.read_csv("clean_train.csv")
data_test = pd.read_csv("clean_test.csv")

In [3]:
%%time 
X_train = []
Y_train = []

curr_sent = 'Sentence: 1'
list_sent = []
list_y = []
for i in range(data_train.shape[0]):
    print("current {}/{}".format(i,data_train.shape[0]), end="\r")
    if data_train["Sentence #"][i] == curr_sent:
        list_sent.append(data_train['Word'][i])
        list_y.append(data_train['Tag'][i])
    else:
        curr_sent = data_train["Sentence #"][i]
        X_train.append(list_sent)
        Y_train.append(list_y)
        list_sent = [data_train['Word'][i]]
        list_y = [data_train['Tag'][i]]
    

CPU times: user 4min 1s, sys: 1min, total: 5min 1s
Wall time: 4min 10s


In [4]:
%%time 
X_test = []
Y_test = []

curr_sent = 'Sentence: 35970'
list_sent = []
list_y = []
for i in range(data_test.shape[0]):
    print("current {}/{}".format(i,data_test.shape[0]), end="\r")
    if data_test["Sentence #"][i] == curr_sent:
        list_sent.append(data_test['Word'][i])
        list_y.append(data_test['Tag'][i])
    else:
        curr_sent = data_test["Sentence #"][i]
        X_test.append(list_sent)
        Y_test.append(list_y)
        list_sent = [data_test['Word'][i]]
        list_y = [data_test['Tag'][i]]

CPU times: user 1min 18s, sys: 18.8 s, total: 1min 37s
Wall time: 1min 21s


In [5]:
X = [[w for w in s] for s in X_train]
Y = [[t for t in s] for s in Y_train]

X_test = [[w for w in s] for s in X_test]
Y_test = [[t for t in s] for s in Y_test]

In [56]:
train_idx, val_idx, _, _ = train_test_split(np.arange(len(X)), np.arange(len(X)),
                                            test_size=0.2, random_state=42)

X_val = [X[i] for i in val_idx]
Y_val = [Y[i] for i in val_idx]

X_train = [X[i] for i in train_idx]
Y_train = [Y[i] for i in train_idx]

In [24]:
prova_train_x = [len(X_train[i]) for i in range(len(X_train))]
prova_train_y = [len(Y_train[i]) for i in range(len(Y_train))]
prova_train_x == prova_train_y

True

In [74]:
for i in range(len(X_train)): 
  for j in range(len(X_train[i])):
    if (X_train[i][j] == '\x85' or X_train[i][j] == '\x94'):
      X_train[i].pop(j)
      Y_train[i].pop(j)

for i in range(len(X_val)): 
  for j in range(len(X_val[i])):
    if (X_val[i][j] == '\x85' or X_val[i][j] == '\x94'):
      X_val[i].pop(j)
      Y_val[i].pop(j)

In [75]:
X_train = [' '.join(X_train[i]) for i in range(len(X_train))]
X_val = [' '.join(X_val[i]) for i in range(len(X_val))]

Y_train = [' '.join(Y_train[i]) for i in range(len(Y_train))]
Y_val = [' '.join(Y_val[i]) for i in range(len(Y_val))]

In [237]:
train_df = pd.DataFrame(np.stack((X_train, Y_train), axis=-1), columns =['sequence', 'tag'])
train_df.head(10)

,sequence,tag
0,Golborne said experts will have to analyze the...,B-per O O O O O O O O O O O O O O O O O O O O ...
1,"The pipeline is run by Petrodar , a conglomera...",O O O O O B-tim O O O O O O B-geo O B-geo I-ge...
2,Election officials in Afghanistan say they hav...,O O O B-geo O O O O O O O O O O O O O O O O O
3,British Foreign Secretary Jack Straw led a bri...,B-gpe O B-per I-per I-per O O O O O B-tim O O ...
4,Turkish government ministers and representativ...,B-gpe O O O O O O O O O O B-tim O O O O O O O ...
5,Cardinal Angelo Sodano said the church should ...,B-per I-per I-per O O O O O O O O O O O O O O O O
6,Los Angeles County fire officials say the blaz...,B-org I-org I-org O O O O O O O O O O O O O O ...
7,Mr. Bakiyev fled Kyrgyzstan after he was toppl...,B-per B-org O B-geo O O O O O B-tim I-tim O O ...
8,His lawyers argue that the books are evidence ...,O O O O O O O O O B-per O O O O O O B-tim I-ti...
9,The opposition in Azerbaijan is calling for pr...,O O O B-gpe O O O O O O O O O O O O O O O O O O O


In [238]:
test_df = pd.DataFrame(np.stack((X_val, Y_val), axis=-1), columns =['sequence', 'tag'])
test_df.head(10)

,sequence,tag
0,Subsistence farming is the dominate form of li...,O O O O O O O O O
1,The peace plan calls on Israel to freeze its s...,O O O O O B-geo O O O O O O O B-org I-org O
2,"Some were freed , but the British government s...",O O O O O O B-gpe O O O O O O O O
3,The Red Cross and the Pentagon have acknowledg...,B-org I-org I-org O O B-org O O O O O O O B-ge...
4,Officials say the doomed boat had set sail fro...,O O O O O O O O O O O O O B-geo O
5,She said authorities must then decide who gets...,O O O O O O O O O O O
6,"Stanley "" Tookie "" Williams has been convicted...",B-per O B-per O B-per O O O O O O O O O O O O ...
7,Bird flu has killed nearly 70 people in Asia s...,O O O O O O O O B-geo B-tim I-tim O O O O O O ...
8,"But Mr. Straw told British radio Wednesday , t...",O B-per I-per O B-gpe O B-tim O O O O O O O O ...
9,"Later Friday , Thai officials said a one-year-...",B-tim I-tim O B-gpe O O O O O O B-geo O O O O ...


In [239]:
train_df.to_csv('train_df_final.csv', index=False)
test_df.to_csv('test_df_final.csv', index=False)

In [240]:
data_train = pd.read_csv("train_df_final.csv")
prova_train_x2 = [data_train['sequence'][i].split() for i in range(len(data_train['sequence']))]
prova_train_y2 = [data_train['tag'][i].split() for i in range(len(data_train['tag']))]

prova_train_x3 = [len(prova_train_x2[i]) for i in range(len(prova_train_x2))]
prova_train_y3 = [len(prova_train_y2[i]) for i in range(len(prova_train_y2))]

prova_train_x3 == prova_train_y3

In [241]:
#define fields
sequence = torchtext.data.Field()
label = torchtext.data.Field(is_target=True)

fields = [('sequence', sequence), ('tag', label)] 

train_data, test_data = torchtext.data.TabularDataset.splits(path ='.', train= 'train_df_final.csv', test= 'test_df_final.csv', format = 'csv', fields = fields, skip_header = True)

In [242]:
#build vocabulary for training dataset using dictionary from torch text 
MAX_VOCAB_SIZE = 35000
#sequence.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
sequence.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
label.build_vocab(train_data)

In [243]:
#length of vocabulary for training dataset (words + 1(unk) + 1(pad))
print(len(label.vocab))
print(len(sequence.vocab))

19
28011


In [244]:
print(label.vocab.stoi)
print(sequence.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fb5e993da60>, {'<unk>': 0, '<pad>': 1, 'O': 2, 'B-geo': 3, 'B-tim': 4, 'B-org': 5, 'I-per': 6, 'B-per': 7, 'I-org': 8, 'B-gpe': 9, 'I-geo': 10, 'I-tim': 11, 'B-art': 12, 'I-art': 13, 'B-eve': 14, 'I-eve': 15, 'B-nat': 16, 'I-gpe': 17, 'I-nat': 18})
defaultdict(<function _default_unk_index at 0x7fb5e993da60>, {'<unk>': 0, '<pad>': 1, 'the': 2, '.': 3, ',': 4, 'in': 5, 'of': 6, 'to': 7, 'a': 8, 'and': 9, 'The': 10, "'s": 11, 'for': 12, 'has': 13, 'is': 14, 'on': 15, 'that': 16, 'with': 17, 'have': 18, 'said': 19, 'was': 20, 'says': 21, 'from': 22, 'by': 23, 'at': 24, 'as': 25, 'say': 26, 'U.S.': 27, 'an': 28, 'are': 29, '"': 30, 'will': 31, 'were': 32, 'his': 33, 'Mr.': 34, 'it': 35, 'government': 36, 'been': 37, 'killed': 38, 'he': 39, 'people': 40, 'after': 41, 'not': 42, 'its': 43, 'President': 44, 'be': 45, 'officials': 46, 'two': 47, 'A': 48, 'also': 49, 'more': 50, 'United': 51, 'country': 52, 'military': 53, 'who': 54, 'than': 55, 'th

In [245]:
#the most common words of the vocabulary for test dataset 
sequence.vocab.freqs.most_common(10), label.vocab.freqs.most_common(10)

([('the', 31414),
  ('.', 28666),
  (',', 19775),
  ('in', 15876),
  ('of', 15767),
  ('to', 13962),
  ('a', 12358),
  ('and', 11969),
  ('The', 6838),
  ("'s", 6571)],
 [('O', 532700),
  ('B-geo', 22384),
  ('B-tim', 12231),
  ('B-org', 12112),
  ('I-per', 10466),
  ('B-per', 10261),
  ('I-org', 10046),
  ('B-gpe', 9396),
  ('I-geo', 4411),
  ('I-tim', 3943)])

In [246]:
#create batches from train and test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64

train_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, test_data),
    sort = False, #don't sort test/validation data
    batch_size = BATCH_SIZE,
    device=device)

In [247]:
#check it works
for batch in train_iterator:
    print(batch)
    break
batch.sequence, batch.tag


[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 35x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 35x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 37x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 37x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 40x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 40x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 47x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 47x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 43x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 43x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 45x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 45x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor

(tensor([[18878,   224,  1021,  ...,  8515,    10,  7560],
         [20096,   313,     2,  ...,  2125,  2000, 15055],
         [  294,    79,  1318,  ...,  7172,  1029,     9],
         ...,
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0'),
 tensor([[7, 9, 2,  ..., 7, 2, 2],
         [6, 2, 2,  ..., 2, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'))

In [248]:
#check it works
for batch in test_iterator:
    print(batch)
    break
batch.sequence, batch.tag


[torchtext.data.batch.Batch of size 64]
	[.sequence]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.tag]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]


(tensor([[13461,    10,   579,  ...,  5312,   148,   224],
         [ 4461,   218,    32,  ...,   214,     2,   338],
         [   14,   281,  1017,  ...,   151,   145,    26],
         ...,
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0'),
 tensor([[2, 2, 2,  ..., 2, 2, 9],
         [2, 2, 2,  ..., 5, 2, 2],
         [2, 2, 2,  ..., 2, 2, 2],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'))

In [249]:
batch.sequence.shape, batch.tag.shape

(torch.Size([46, 64]), torch.Size([46, 64]))

In [278]:
#model
class RNN(torch.nn.Module):
    def __init__(self, vocab_dim, text, hidden_dim=256, output_dim=len(label.vocab), dropout=0., embedding_dim=100, bidirectional=True, num_layers=2):
        super().__init__()
        vocab = text.vocab
        self.embedding = torch.nn.Embedding(vocab_dim, embedding_dim)
        #self.embedding.weight.data.copy_(vocab.vectors)
        self.rnn = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = torch.nn.Linear(hidden_dim*2 if bidirectional else hidden_dim, output_dim)
        
    def forward(self, sequence):

        #question = [sent len, batch size]        
        embedded = self.embedding(sequence)
    
        #embedded = [sent len, batch size, emb dim]        
        output, hidden = self.rnn(embedded)   
        
        #output = [sent len, batch size, hid dim]  
        L, B, H = output.shape
        output = self.fc(output.view(-1, H)) #B*L, H
        output = output.view(L, B, -1) #L, B, output_dim

        #output = [sent len, batch size, output_dim]
        L, B, C = output.shape  

        return output.view(B,C,L)

In [279]:
#model to gpu
model = RNN(vocab_dim=len(sequence.vocab), text=sequence)
model.to(device)

RNN(
  (embedding): Embedding(28011, 100)
  (rnn): LSTM(100, 256, num_layers=2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=19, bias=True)
)

In [280]:
#check dimensions of output
output = model(batch.sequence.to(device))
output.shape

torch.Size([64, 19, 46])

In [282]:
model = RNN(vocab_dim=len(sequence.vocab), text=sequence)
model.to(device)

epochs = 2000
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=10**(-3))

for epoch in range(1, epochs+1):
  loss_values_train = []
  loss_values_test = []

  acc_values_train = []
  acc_values_test = []

  for batch_train, batch_test in zip(train_iterator, test_iterator):
    model.train()
    output_train = model(batch_train.sequence.to(device)) 
    loss_train = criterion(output_train, torch.reshape(batch_train.tag, (batch_train.tag.shape[1], batch_train.tag.shape[0])).long().to(device))
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()     
    loss_values_train.append(loss_train.item())    
    output_train = torch.argmax(output_train, dim=1)
    accuracy_sum_train = (torch.reshape(output_train, (output_train.shape[1], output_train.shape[0])) == batch_train.tag.to(device).float()).sum()
    accuracy_train = (accuracy_sum_train.cpu().detach().numpy())/(batch_train.tag.shape[0]*batch_train.tag.shape[1])
    acc_values_train.append(accuracy_train)
  
    model.eval()
    with torch.no_grad():
      output_test = model(batch_test.sequence.to(device)) 
      loss_test = criterion(output_test, torch.reshape(batch_test.tag, (batch_test.tag.shape[1], batch_test.tag.shape[0])).to(device))
      loss_values_test.append(loss_test.item()) 
      output_test = torch.argmax(output_test, dim=1)
      accuracy_sum_test = (torch.reshape(output_test, (output_test.shape[1], output_test.shape[0])) == batch_test.tag.to(device).float()).sum()
      accuracy_test = (accuracy_sum_test.cpu().detach().numpy())/(batch_test.tag.shape[0]*batch_test.tag.shape[1])
      acc_values_test.append(accuracy_test)
  
  print('Epoch: ' + str(epoch) + 
        ' Train loss: ' + str(np.mean(loss_values_train)) + 
        ' Test loss: ' + str(np.mean(loss_values_test)) +
        ' Train accuracy: ' + str(accuracy_train) +
        ' Test accuracy: ' + str(accuracy_test))

Epoch: 1 Train loss: 2.944789660715424 Test loss: 2.94457280108359 Train accuracy: 0.04382621951219512 Test accuracy: 0.0708502024291498
Epoch: 2 Train loss: 2.9442469377433302 Test loss: 2.944220998645884 Train accuracy: 0.060546875 Test accuracy: 0.06781376518218624
Epoch: 3 Train loss: 2.943707691884674 Test loss: 2.9439967649172893 Train accuracy: 0.04517663043478261 Test accuracy: 0.07793522267206478
Epoch: 4 Train loss: 2.9437430115927636 Test loss: 2.943874610208832 Train accuracy: 0.073046875 Test accuracy: 0.08805668016194332
Epoch: 5 Train loss: 2.943742157083697 Test loss: 2.9436910806503973 Train accuracy: 0.052083333333333336 Test accuracy: 0.09817813765182186
Epoch: 6 Train loss: 2.942550287837476 Test loss: 2.9424368027037224 Train accuracy: 0.06815159574468085 Test accuracy: 0.08299595141700405
Epoch: 7 Train loss: 2.9370432743983987 Test loss: 2.9370352884309483 Train accuracy: 0.0764627659574468 Test accuracy: 0.11842105263157894
Epoch: 8 Train loss: 2.930080612148859

KeyboardInterrupt: ignored